In [3]:
import torch
import pprint
import evaluate
import numpy as np

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)

In [4]:
import numpy as np
import pandas as pd
df = pd.read_csv('./final_data.csv')
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,preprocessed_text,Id.1,Category
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,summarize: i have bought several of the vitali...,1.0,Pet Food
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,summarize: product arrived labeled as jumbo sa...,2.0,Snacks
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,summarize: this is a confection that has been ...,3.0,Confectionery
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,summarize: if you are looking for the secret i...,4.0,Beverages
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,summarize: great taffy at a great price. there...,5.0,Pet Food
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,B000P41A28,A3A63RACXR1XIL,"A. Boodhoo ""deaddodo""",10,15,1,1204502400,constipation,we switched from the advance similac to the or...,summarize: we switched from the advance simila...,NaN,product review
9996,9997,B000P41A28,A5VVRGL8JA7R,Adam,2,3,5,1306368000,Constipation Not A Problem if...,"Like the bad reviews say, the organic formula ...","summarize: like the bad reviews say, the organ...",NaN,product review
9997,9998,B000P41A28,A2TGDTJ8YCU6PD,geena77,0,0,5,1347494400,Love this formula!,I wanted to solely breastfeed but was unable t...,summarize: i wanted to solely breastfeed but w...,NaN,product review
9998,9999,B000P41A28,AUV4GIZZE693O,"Susan Coe ""sueysis""",1,2,5,1203638400,very convenient,i love the fact that i can get this delieved t...,summarize: i love the fact that i can get this...,NaN,product review


In [5]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(df, test_size=0.2, random_state=42)

In [18]:
dataset_train, dataset_val = train_test_split(dataset_train, test_size=0.2, random_state=42)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
tokenizer = T5Tokenizer.from_pretrained("./Finetuned_t5/tokenizer")
model = T5ForConditionalGeneration.from_pretrained("./Finetuned_t5/model").to(device)

In [10]:
rouge = evaluate.load("rouge")

In [19]:
dataset_train.reset_index(drop=True, inplace=True)
dataset_test.reset_index(drop=True, inplace=True)
dataset_val.reset_index(drop=True, inplace=True)

In [20]:
dataset_train

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,preprocessed_text,Id.1,Category
0,2580,B000EGX2EG,A1K5VQRQGT4244,Ninjamama,2,2,5,1162684800,Mrs. Mays almond crunch,"These are fresh, crunchy, natural, and healthy...","summarize: these are fresh, crunchy, natural, ...",NaN,product review
1,7129,B004K30HO2,A2I5TTL2JYS2RA,jerry joe,0,0,3,1332720000,Patience is the key!,I was looking for a simple and inexpensive way...,summarize: i was looking for a simple and inex...,NaN,product review
2,4083,B001EW5YQS,A3EISQCOUUSWY2,C-Rain,2,2,5,1304812800,Works Awesome,"Read a few reviews, some good and some bad, an...","summarize: read a few reviews, some good and s...",NaN,product review
3,2551,B000ED9LDU,A26YHIRPMZYZDU,"Pankaj Gupta ""Pankaj Gupta""",3,3,5,1220832000,Good product,Good product. But remember that you need some ...,summarize: good product. but remember that you...,NaN,product review
4,4927,B0009JMW1C,A2WW1USVOO6UXN,Love The Tea,0,1,5,1257638400,Absolute Recommend This Product,I absolutely recommend this product to anyone ...,summarize: i absolutely recommend this product...,NaN,product review
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,8872,B001EQ5EQE,A2GFDGHAODEBX8,Jessica,1,1,5,1329782400,excellent coffee,We love this coffee alone or mixed with caffei...,summarize: we love this coffee alone or mixed ...,NaN,product review
6396,9827,B001EQ5CK2,ANGRTZCX57VD7,Katherine,0,0,5,1301788800,This is the best coffee,"You get a fresh, bold, thick flavor from this ...","summarize: you get a fresh, bold, thick flavor...",NaN,product review
6397,5269,B0012YEKCW,A3MNXA1HD7S3VN,RogerDreamNC,0,0,5,1316044800,Cat loves it!,Our cat loves to eat this grass and it was so ...,summarize: our cat loves to eat this grass and...,NaN,product review
6398,9667,B001LXYA5Q,A5HG0O6UAV3NL,M. Meyer,3,3,5,1240012800,Miracle Berry Fruit Tablets,A friend referred me to these and told me that...,summarize: a friend referred me to these and t...,NaN,product review


In [13]:
def preprocess_data(row, tokenizer, max_input_length=512, max_target_length=50):
    """
    Tokenize a single row for T5 training.
    """
    # Get the 'Text' and 'Summary' columns directly from the row (Series)
    inputs = row["preprocessed_text"] + "Category:" + row["Category"] # Access the 'Text' column
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    labels = tokenizer(row["Summary"], max_length=max_target_length, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [21]:
tokenized_train = dataset_train.apply(
    lambda row: preprocess_data(row, tokenizer),
    axis=1
)

tokenized_test = dataset_test.apply(
    lambda row: preprocess_data(row, tokenizer),
    axis=1
)

tokenized_val = dataset_val.apply(
    lambda row: preprocess_data(row, tokenizer),
    axis=1
)

In [22]:
tokenized_train

,0
0,"[input_ids, attention_mask, labels]"
1,"[input_ids, attention_mask, labels]"
2,"[input_ids, attention_mask, labels]"
3,"[input_ids, attention_mask, labels]"
4,"[input_ids, attention_mask, labels]"
...,...
6395,"[input_ids, attention_mask, labels]"
6396,"[input_ids, attention_mask, labels]"
6397,"[input_ids, attention_mask, labels]"
6398,"[input_ids, attention_mask, labels]"


In [23]:
tokenized_train[0]

{'input_ids': [21603, 10, 175, 33, 1434, 6, 29334, 6, 793, 6, 11, 1695, 12751, 25, 54, 3, 60, 7, 15, 138, 11, 453, 16, 8, 443, 6, 22854, 15, 139, 529, 2886, 19168, 42, 147, 5870, 7, 6, 42, 4153, 128, 16, 8, 1082, 3074, 5, 4979, 11, 3326, 5, 18610, 6066, 651, 10, 15892, 1132, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [27]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    # Decode predictions and references
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    # Compute ROUGE scores
    result = rouge.compute(predictions=pred_str, references=label_str)
    return {key: value.mid.fmeasure * 100 for key, value in result.items()}

In [40]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="t5_summarization",  # Where to save the model
    evaluation_strategy="steps",    # Evaluate after every `eval_steps`
    eval_steps=50,                 # Number of steps between evaluations
    logging_dir="./logs",           # Where to save logs
    logging_steps=100,              # Log training stats every 500 steps
    save_steps=200,                # Save model checkpoints every 1000 steps
    save_total_limit=5,             # Keep only the last two saved models
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,   # Batch size for evaluation
    num_train_epochs=15,             # Number of epochs
    learning_rate=5e-5,             # Learning rate
    weight_decay=0.01,              # Weight decay for regularization
    warmup_steps=500,               # Number of warmup steps for learning rate scheduler
    fp16=torch.cuda.is_available(),  # Use FP16 for faster training if GPU is available
    report_to="none"                # Disable reporting to external tools (e.g., wandb)
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-40-4ad0ce5c6ef9>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [41]:
history = trainer.train()

Step,Training Loss,Validation Loss
50,No log,0.490830
100,0.476900,0.490908
150,0.476900,0.491192
200,0.477500,0.491469
250,0.477500,0.491440
300,0.451500,0.492695
350,0.451500,0.492302
400,0.497300,0.492018
450,0.497300,0.492232
500,0.450700,0.493045


In [39]:

torch.cuda.empty_cache()


In [42]:
trainer.save_model("./Finetuned_T5/model")
tokenizer.save_pretrained("./Finetuned_T5/tokenizer")

('./Finetuned_T5/tokenizer/tokenizer_config.json',
 './Finetuned_T5/tokenizer/special_tokens_map.json',
 './Finetuned_T5/tokenizer/spiece.model',
 './Finetuned_T5/tokenizer/added_tokens.json')

In [43]:
tokenizer = T5Tokenizer.from_pretrained("./Finetuned_T5/tokenizer")
model = T5ForConditionalGeneration.from_pretrained("./Finetuned_T5/model").to(device)

In [44]:
def summarize_text_finetuned(text, max_input_length=512, max_output_length=15):
    """
    Summarizes input text using a T5 model.
    """
    # Add the "summarize:" prefix for T5
    input_text = text.strip()

    # Tokenize the input text
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=max_input_length).to(device)

    # Generate the summary
    outputs = model.generate(inputs, max_length=max_output_length, min_length=2, length_penalty=2.0, num_beams=4)

    # Decode the summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [45]:
dataset_test["generated_summary"] = dataset_test["preprocessed_text"].apply(summarize_text_finetuned)

In [46]:
results = rouge.compute(
    predictions=dataset_test["generated_summary"].tolist(),
    references=dataset_test["Summary"].tolist()
)
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

rouge1: 0.1421
rouge2: 0.0477
rougeL: 0.1370
rougeLsum: 0.1370
